Download data van ENTSO-E

In [ ]:
import pandas as pd
from entsoe import EntsoePandasClient
import time

# Insert your API key here once you get it
API_KEY = '82aa28d4-59f3-4e3a-b144-6659aa9415b5'

# Initialize ENTSO-E client
client = EntsoePandasClient(api_key=API_KEY)

# Define parameters
country_code = 'NL'
neighboring_countries = ['BE', 'DE', 'GB', 'DK', 'NO']  # Pas dit aan op basis van de relevante buren
years = [2022, 2023, 2024]  # List of years to fetch

# Data storage
all_data = []

# Function to fetch data with retries
def fetch_with_retries(fetch_func, *args, retries=3, delay=5, **kwargs):
    for attempt in range(retries):
        try:
            return fetch_func(*args, **kwargs)
        except Exception as e:
            print(f"Error: {e}, retrying in {delay} seconds...")
            time.sleep(delay)
    raise Exception(f"Failed to fetch data after {retries} attempts")

# Loop through each year and fetch data separately
for year in years:
    start = pd.Timestamp(f'{year}-01-01', tz='Europe/Amsterdam')
    end = pd.Timestamp(f'{year+1}-01-01', tz='Europe/Amsterdam')  # Exclusive end

    print(f"Fetching load data for {year}...")
    yearly_load = fetch_with_retries(client.query_load, country_code, start=start, end=end).squeeze()  # Convert to 1D Series

    print(f"Fetching load forecast for {year}...")
    yearly_load_forecast = fetch_with_retries(client.query_load_forecast, country_code, start=start, end=end).squeeze()  # Convert to 1D Series

    print(f"Fetching price data for {year}...")
    yearly_price = fetch_with_retries(client.query_day_ahead_prices, country_code, start=start, end=end).squeeze()  # Convert to 1D Series

    # Fetch cross-border flows
    flow_data = {}
    for neighbor in neighboring_countries:
        print(f"Fetching cross-border flow from {neighbor} to {country_code} for {year}...")
        yearly_flow_to = fetch_with_retries(client.query_crossborder_flows, country_code_from=neighbor, 
                                            country_code_to=country_code, start=start, end=end).squeeze()  # Convert to 1D Series
        flow_data[f'Flow_{neighbor}_to_{country_code}'] = yearly_flow_to

        print(f"Fetching cross-border flow from {country_code} to {neighbor} for {year}...")
        yearly_flow_from = fetch_with_retries(client.query_crossborder_flows, country_code_from=country_code, 
                                              country_code_to=neighbor, start=start, end=end).squeeze()  # Convert to 1D Series
        flow_data[f'Flow_{country_code}_to_{neighbor}'] = yearly_flow_from

    # Merge all data
    if not yearly_load.empty and not yearly_price.empty:
        df = pd.DataFrame({'Load': yearly_load, 'Price': yearly_price})
        for col_name, flow_series in flow_data.items():
            if not flow_series.empty:
                df[col_name] = flow_series

        # Store yearly data
        all_data.append(df)
    else:
        print(f"No data for year {year}")

# Concatenate all years into one DataFrame if there is data
if all_data:
    final_data = pd.concat(all_data)
    # Save to CSV
    final_data.to_csv('electricity_data_nl_2022_2024.csv')
    print("Data saved successfully!")
else:
    print("No data to save.")

Read excel file + data per uur

In [ ]:
import pandas as pd
from entsoe import EntsoePandasClient
import time

try:
    # Load the existing CSV file
    df = pd.read_csv('electricity_data_nl_2022_2024.csv', index_col=0, parse_dates=True)
    print("CSV file loaded successfully!")

    # Ensure the index is datetime with UTC
    df.index = pd.to_datetime(df.index, utc=True)
    print("Index converted to datetime with UTC!")

    # Resample the data to hourly frequency and calculate the mean for each hour
    df_hourly = df.resample('h').mean()
    print("Data resampled to hourly frequency!")
    
    # Save the resampled data to a new CSV file
    df_hourly.to_csv('electricity_data_nl_2022_2024_hourly.csv')
    print("Hourly data with flow saved successfully!")

except FileNotFoundError:
    print("Error: The file 'electricity_data_nl_2022_2024.csv' does not exist.")
except pd.errors.EmptyDataError:
    print("Error: The file 'electricity_data_nl_2022_2024.csv' is empty.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


In [ ]:
Nieuwe variabelen flow toevoegen (per land en totaal)

In [14]:
neighboring_countries = ['BE', 'DE', 'GB', 'DK', 'NO']

for neighbor in neighboring_countries:
    df_hourly[f'Flow_{neighbor}'] = df_hourly[f'Flow_{neighbor}_to_NL'] - df_hourly[f'Flow_NL_to_{neighbor}']

df_hourly['Total_Flow'] = df_hourly['Flow_BE'] + df_hourly['Flow_DE'] + df_hourly['Flow_GB'] + df_hourly['Flow_DK'] + df_hourly['Flow_DK']
print(df_hourly)

df_hourly.to_csv('electricity_data_nl_2022_2024_hourly_flow.csv')
print("Hourly data saved successfully!")

                               Load   Price  Flow_BE_to_NL  Flow_NL_to_BE  \
2021-12-31 23:00:00+00:00  10374.00  124.70            0.0          620.0   
2022-01-01 00:00:00+00:00  10249.75  124.70            0.0          728.0   
2022-01-01 01:00:00+00:00   9907.00  134.00            0.0         1070.0   
2022-01-01 02:00:00+00:00   9782.50   58.80            0.0          344.0   
2022-01-01 03:00:00+00:00   9589.75   37.67            0.0          154.0   
...                             ...     ...            ...            ...   
2024-12-31 19:00:00+00:00  14059.25   59.90            0.0         1599.0   
2024-12-31 20:00:00+00:00  13267.50   18.20            0.0          959.0   
2024-12-31 21:00:00+00:00  12692.00    9.06            0.0          522.0   
2024-12-31 22:00:00+00:00  12304.75    0.52            0.0          373.0   
2024-12-31 23:00:00+00:00       NaN   13.62            NaN            NaN   

                           Flow_DE_to_NL  Flow_NL_to_DE  Flow_GB_to_NL  \
2